In [ ]:
!pip install tensorboardX
import time
import pickle
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score,classification_report, precision_recall_fscore_support
from model import BiModel, Model, MaskedNLLLoss
from dataloader import IEMOCAPDataset
from IEMOCAP_utils import *
np.random.seed(1234)

DATA_PATH='../../dataset_raw_pkl/iemocap_features/IEMOCAP_features_raw.pkl'
modelName='dialogueRnn_2020-04-08 20:40:11.pth'

In [ ]:
no_cuda=False           # 'does not use GPU'
lr=0.0001               # 'learning rate'
l2=0.00001              # 'L2 regularization weight'
rec_dropout=0.1         # 'rec_dropout rate'
dropout=0.1             # 'dropout rate'
batch_size=30           # 'batch size'
n_epochs = 20            #'number of epochs'
class_weight=True       # 'class weight'
active_listener=False   # 'active listener'
attention='general'     # 'Attention type'
tensorboard=False       # 'Enables tensorboard log'
D_a = 100               # concat attention
D_m = 100
D_g = 150
D_p = 150
D_e = 100
D_h = 100
n_classes  = 6


In [ ]:
train_loader, valid_loader, test_loader =get_IEMOCAP_loaders(DATA_PATH,valid=0.0,batch_size=batch_size,num_workers=2)

In [ ]:
# Initialization of Model
cuda = init_cuda(no_cuda)

model = BiModel(D_m, D_g, D_p, D_e, D_h,
                n_classes=n_classes,
                listener_state=active_listener,
                context_attention=attention,
                dropout_rec=rec_dropout,
                dropout=dropout)
if cuda:
    model.cuda()

optimizer = optim.Adam(model.parameters(),
                        lr=lr,
                        weight_decay=l2)

In [ ]:
# Train the model and save in the model directory

if tensorboard:
    from tensorboardX import SummaryWriter

writer = SummaryWriter()

best_loss, best_label, best_pred, best_mask = None, None, None, None

for e in range(n_epochs):
    start_time = time.time()
    train_loss, train_acc, _,_,_,train_fscore,_= train_model(model,tensorboard,train_loader, e, cuda, optimizer)
    valid_loss, valid_acc, _,_,_,val_fscore,_= eval_model(model,valid_loader,cuda)
    test_loss, test_acc, test_label, test_pred, test_mask, test_fscore, attentions = eval_model(model,test_loader,cuda)

    if best_loss == None or best_loss > test_loss:
        best_loss, best_label, best_pred, best_mask, best_attn =\
                test_loss, test_label, test_pred, test_mask, attentions

    if tensorboard:
        writer.add_scalar('test: accuracy/loss',test_acc/test_loss,e)
        writer.add_scalar('train: accuracy/loss',train_acc/train_loss,e)
    print('epoch {} train_loss {} train_acc {} train_fscore{}'.format(e+1, train_loss, train_acc, train_fscore)); 
    print('valid_loss {} valid_acc {} val_fscore{}'.format(valid_loss, valid_acc, val_fscore))
    print('test_loss {} test_acc {} test_fscore {} time {}'.format(test_loss, test_acc, test_fscore, round(time.time()-start_time,2)))

if tensorboard:
    writer.close()

# Save model 
# save_model(model)
print('Test performance..')
print('Loss {} accuracy {}'.format(best_loss,
                                  round(accuracy_score(best_label,best_pred,sample_weight=best_mask)*100,2)))
print(classification_report(best_label,best_pred,sample_weight=best_mask,digits=4))
print(confusion_matrix(best_label,best_pred,sample_weight=best_mask))

In [ ]:
# Test the trained model
model = load_model(modelName)
test_loss, test_acc, test_label, test_pred, test_mask, test_fscore, attentions = eval_model(model, test_loader,cuda)
print('Test performance..')
print('Loss {} accuracy {}'.format(test_loss,
                                  round(accuracy_score(test_label,test_pred,sample_weight=test_mask)*100,2)))
print(classification_report(test_label,test_pred,sample_weight=test_mask,digits=4))
print(confusion_matrix(test_label,test_pred,sample_weight=test_mask))